# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [ ]:
# импорт библиотек
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")
import math
from statistics import mean

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error


import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# посмотр, где находится каталог с файлами на COLAB
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# получение доступа к каталогу и уточнение названия папок
import os
os.listdir('/content/drive/My Drive/Colab Notebooks/Яндекс/Проект 7 Выбор локации для скважины')

['geo_data_0.csv', 'geo_data_2.csv', 'geo_data_1.csv', 'ПРАВКА', 'GitHub']

In [ ]:
# загрузка данных
data_0 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Яндекс/Проект 7 Выбор локации для скважины/geo_data_0.csv')
data_1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Яндекс/Проект 7 Выбор локации для скважины/geo_data_1.csv')
data_2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Яндекс/Проект 7 Выбор локации для скважины/geo_data_2.csv')

In [ ]:
# просмотр типов данных и кол-ва ненулевых значений
data_0.info()
print('______________________________________')
data_1.info()
print('______________________________________')
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
______________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
______________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000

In [ ]:
# подсчет нулевых значений
data_0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [ ]:
data_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [ ]:
data_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [ ]:
# конкатенация таблиц
data_all = pd.concat([data_0] + [data_1] + [data_2], axis=1)

In [ ]:
data_all

,id,f0,f1,f2,product,id,f0,f1,f2,product,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062,kBEdx,-15.001348,-8.276000,-0.005876,3.179103,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,2acmU,1.334711,-0.340164,4.365080,73.037750,62mP7,14.272088,-3.475083,0.999183,26.953261,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,409Wp,1.022732,0.151990,1.419926,85.265647,vyE1P,6.263187,-5.948386,5.001160,134.766305,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,iJLyR,-0.032172,0.139033,2.978566,168.620776,KcrkZ,-13.081196,-11.506057,4.999415,137.945408,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,Xdl7t,1.988431,0.155413,4.751769,154.036647,AHL4O,12.702195,-8.147433,5.004363,134.766305,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026,QywKC,9.535637,-6.878139,1.998296,53.906522,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,QKivN,1.392429,-0.382606,1.273912,122.346843,ptvty,-10.160631,-12.558096,5.005581,137.945408,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443,09gWa,-7.378891,-3.084104,4.998651,137.945408,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,7kl59,0.998163,-0.528582,1.583869,74.040764,rqwUm,0.665714,-6.152593,1.000146,30.132364,nmxp2,-2.419896,2.417221,-5.548444,51.795253


In [ ]:
# подсчет доли дубликатов
print(data_0.duplicated().sum() / len(data_0) * 100)
print(data_1.duplicated().sum() / len(data_1) * 100)
print(data_2.duplicated().sum() / len(data_2) * 100)

0.0
0.0
0.0


In [ ]:
# вывод количества уникальных значений 'id'
len(data_0['id'].unique())


99990

In [ ]:
# вывод количества уникальных значений 'id'
len(data_1['id'].unique())

99996

In [ ]:
# вывод количества уникальных значений 'id'
len(data_2['id'].unique())

99996

In [ ]:
# сортировка по количеству повторений уникальных значений 'id'
data_0['id'].value_counts(ascending=False).head(12)

fiKDv    2
QcMuo    2
AGS9W    2
Tdehs    2
74z30    2
A5aEY    2
bsk9y    2
TtcGQ    2
HZww2    2
bxg6G    2
iAaqy    1
PQ2MT    1
Name: id, dtype: int64

In [ ]:
data_0_dupl = data_0.query('id == ["fiKDv", "QcMuo", "AGS9W", "Tdehs", "74z30", "A5aEY", "bsk9y", "TtcGQ", "HZww2", "bxg6G"]')

In [ ]:
data_0_dupl

,id,f0,f1,f2,product
931,HZww2,0.755284,0.368511,1.863211,30.681774
1364,bxg6G,0.411645,0.856830,-3.653440,73.604260
1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502
3389,A5aEY,-0.039949,0.156872,0.209861,89.249364
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
16633,fiKDv,0.157341,1.028359,5.585586,95.817889
21426,Tdehs,0.829407,0.298807,-0.049563,96.035308
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798
42529,AGS9W,1.454747,-0.479651,0.683380,126.370504
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205


In [ ]:
# сортировка по количеству повторений уникальных значений 'id'
data_1['id'].value_counts(ascending=False).head(6)

wt4Uk    2
LHZR0    2
5ltQ6    2
bfPNe    2
kBEdx    1
lAVJZ    1
Name: id, dtype: int64

In [ ]:
data_1_dupl = data_1.query('id == ["wt4Uk", "LHZR0", "5ltQ6", "bfPNe"]')

In [ ]:
data_1_dupl

,id,f0,f1,f2,product
1305,LHZR0,11.170835,-1.945066,3.002872,80.859783
2721,bfPNe,-9.494442,-5.463692,4.006042,110.992147
5849,5ltQ6,-3.435401,-12.296043,1.999796,57.085625
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
47591,wt4Uk,-9.091098,-8.109279,-0.002314,3.179103
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044


In [ ]:
# сортировка по количеству повторений уникальных значений 'id'
data_2['id'].value_counts(ascending=False).head(6)

VF7Jo    2
KUPhW    2
xCHr8    2
Vcm5J    2
fwXo0    1
B9hou    1
Name: id, dtype: int64

In [ ]:
data_2_dupl = data_2.query('id == ["VF7Jo", "KUPhW", "xCHr8", "Vcm5J"]')

In [ ]:
data_2_dupl

,id,f0,f1,f2,product
11449,VF7Jo,2.122656,-0.858275,5.746001,181.716817
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641
44378,Vcm5J,-1.229484,-2.439204,1.222909,137.968290
45404,KUPhW,0.231846,-1.698941,4.990775,11.716299
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
55967,KUPhW,1.211150,3.176408,5.543540,132.831802
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572


**В результате поиска неявных дубликатов, установлено:**

1. В регионе "data_0" - 10 пар с одинаковыми "id".
2. В регионах "data_1", "data_2" по 4 пары с одинаковыми "id".
3. Так как, прослеживается парность и параметры меняются либо в сторону уменьшения, либо в сторону увеличения, можно предположить, что это результат доразветки скважин. По итогам доразведки уточнялись параметры скважины и ее объём запасов. Исходя из этого, удалим первые (неуточненные данные), а последние записи (по индексу) в соответствующих "id" оставим для дальнейших расчетов.

In [ ]:
data_0 = data_0.query("index not in [931, 1364, 1949, 3389, 16633, 21426, 42529, 60140, 64022, 89582]")
data_1 = data_1.query("index not in [1305, 2721, 5849, 47591]")
data_2 = data_2.query("index not in [11449, 28039, 44378, 45404]")

In [ ]:
# проверка результатов удаления дубликатов доразведанных скважин
data_0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99990 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99990 non-null  object 
 1   f0       99990 non-null  float64
 2   f1       99990 non-null  float64
 3   f2       99990 non-null  float64
 4   product  99990 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


In [ ]:
# проверка результатов удаления дубликатов доразведанных скважин
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99996 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99996 non-null  object 
 1   f0       99996 non-null  float64
 2   f1       99996 non-null  float64
 3   f2       99996 non-null  float64
 4   product  99996 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


In [ ]:
# проверка результатов удаления дубликатов доразведанных скважин
data_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99996 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       99996 non-null  object 
 1   f0       99996 non-null  float64
 2   f1       99996 non-null  float64
 3   f2       99996 non-null  float64
 4   product  99996 non-null  float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


In [ ]:
# сброс индексов
data_0 = data_0.reset_index(drop = True)
data_1 = data_1.reset_index(drop = True)
data_2 = data_2.reset_index(drop = True)

In [ ]:
# просмотр описательной статистики
data_0.describe()

,f0,f1,f2,product
count,99990.000000,99990.000000,99990.000000,99990.000000
mean,0.500412,0.250149,2.502656,92.499482
std,0.871867,0.504434,3.248155,44.289431
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072683,-0.200877,0.287887,56.496838
50%,0.502340,0.250252,2.516005,91.849972
75%,1.073626,0.700654,4.714970,128.564269
max,2.362331,1.343769,16.003790,185.364347


In [ ]:
# просмотр описательной статистики
data_1.describe()

,f0,f1,f2,product
count,99996.000000,99996.000000,99996.000000,99996.000000
mean,1.141450,-4.796493,2.494551,68.825232
std,8.965922,5.119900,1.703579,45.944649
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298525,-8.267985,1.000021,26.953261
50%,1.153190,-4.813005,2.011479,57.085625
75%,8.621015,-1.332704,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [ ]:
# просмотр описательной статистики
data_2.describe()

,f0,f1,f2,product
count,99996.000000,99996.000000,99996.000000,99996.000000
mean,0.001995,-0.002034,2.495132,95.000425
std,1.732054,1.730424,3.473453,44.748112
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174805,0.130359,59.452094
50%,0.009411,-0.009433,2.484236,94.925613
75%,1.158477,1.163760,4.858720,130.590822
max,7.238262,7.844801,16.739402,190.029838


## Вывод

**В результате подготовки данных, установлено:**

1. Пропуски отсутствуют.
2. Тип данных во всех столбцах float64, кроме столбца "id" - object.
3. В регионе "data_0" - 10 пар с одинаковыми "id".
4. В регионах "data_1", "data_2" по 4 пары с одинаковыми "id".
5. Так как, прослеживается парность и параметры меняются либо в сторону уменьшения, либо в сторону увеличения, можно предположить, что это результат доразветки скважин. По итогам доразведки уточнялись параметры скважины и ее объём запасов. Исходя из этого, удалим первые (неуточненные данные), а последние записи (по индексу) в соответствующих "id" оставим для дальнейших расчетов.
6. Просмотр описательной статистики показал, что самый большой средний объём запасов в скважинах региона "data_2".
7. Стандартное отклонение в объёмах запасов в скважинах по трем регионам однородно и колеблется в пределах от 44 до 45 тыс. баррелей.
8. Максимальное значение объёма запасов в скважине, также зафиксировано в регионе "data_2" и составляет порядка 190 тыс. баррелей.
9. Аномально высоких или низких значений не обнаружено.
10. Дубликаты в данных не обнаружены.
11. Неявные дубликаты устранены.

## Обучение и проверка модели

In [ ]:
# функция вывода на печать предсказанного и истинного среднего запаса сырья, а также параметров, оценивающих качество модели
def param_model(data):
  # удаление столбцов из признаков
  # назначение целевого признака
  features = data.drop(['id', 'product'], axis=1)
  target = data['product']
  # деление данных на обучающую и валидационную выборки в соотношении 75:25
  features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
  # обучение моли линейной регрессии на данных региона
  # вывод предсказанного среднего запаса сырья
  # вывод метрик качества модели линейной регрессии
  model = LinearRegression()
  model.fit(features_train, target_train)
  predicted_valid = model.predict(features_valid)
  mse = mean_squared_error(target_valid, predicted_valid)
  
  print("Предсказанный средний запас сырья =", round(predicted_valid.mean(), 1))
  print()
  print("RMSE =", round(mse ** 0.5, 1))
  print("MSE =", round(mse, 1))
  print("R2 =", round(r2_score(target_valid, predicted_valid), 1))
  print("MAE =", round(mean_absolute_error(target_valid, predicted_valid), 1))
  print()
  print("Истинное значение среднего запаса сырья =", round(data['product'].mean(), 1))

### Модель для региона "data_0"

In [ ]:
param_model(data_0)

Предсказанный средний запас сырья = 92.6

RMSE = 37.9
MSE = 1438.4
R2 = 0.3
MAE = 31.2

Истинное значение среднего запаса сырья = 92.5


### Вывод

**Анализ работы модели линейной регрессии для региона "data_0", показал:**

1. Предсказанный средний запас сырья = 92.6
2. Истинное значение среднего запаса сырья = 92.5
3. Модель демонстрирует хорошие результаты: предсказанное и истинное значения равны при округлении до целых (93=93)
4. RMSE = 37.9

### Модель для региона "data_1"

In [ ]:
param_model(data_1)

Предсказанный средний запас сырья = 68.5

RMSE = 0.9
MSE = 0.8
R2 = 1.0
MAE = 0.7

Истинное значение среднего запаса сырья = 68.8


### Вывод

**Анализ работы модели линейной регрессии для региона "data_1", показал:**

1. Предсказанный средний запас сырья = 68.5
2. Истинное значение среднего запаса сырья = 68.8
3. Модель демонстрирует хорошие результаты: предсказанное и истинное значения равны при округлении до целых (69=69)
4. RMSE = 0.9

### Модель для региона "data_2"

In [ ]:
param_model(data_2)

Предсказанный средний запас сырья = 94.9

RMSE = 40.2
MSE = 1614.9
R2 = 0.2
MAE = 32.9

Истинное значение среднего запаса сырья = 95.0


### Вывод

**Анализ работы модели линейной регрессии для региона "data_2", показал:**

1. Предсказанный средний запас сырья = 94.9
2. Истинное значение среднего запаса сырья = 95.0
3. Модель демонстрирует хорошие результаты: предсказанное и истинное значения равны при округлении до целых (95=95)
4. RMSE = 40.2

## Вывод

**А) Анализ работы модели линейной регрессии для региона "data_0", показал:**

1. Предсказанный средний запас сырья = 92.6
2. Истинное значение среднего запаса сырья = 92.5
3. Модель демонстрирует хорошие результаты: предсказанное и истинное значения равны при округлении до целых (93=93)
4. RMSE = 37.9

**Б) Анализ работы модели линейной регрессии для региона "data_1", показал:**

1. Предсказанный средний запас сырья = 68.5
2. Истинное значение среднего запаса сырья = 68.8
3. Модель демонстрирует хорошие результаты: предсказанное и истинное значения равны при округлении до целых (69=69)
4. RMSE = 0.9

**В) Анализ работы модели линейной регрессии для региона "data_2", показал:**

1. Предсказанный средний запас сырья = 94.9
2. Истинное значение среднего запаса сырья = 95.0
3. Модель демонстрирует хорошие результаты: предсказанное и истинное значения равны при округлении до целых (95=95)
4. RMSE = 40.2


## Подготовка к расчёту прибыли

In [ ]:
# дано по условию
EXPLORE_POINTS =500
BEST_POINTS = 200
BUDGET = 10 * 10 ** 9
INCOME_ONE_THOUSAND_BARREL = 450000
LOSS_PROBABILITY = 0.025

In [ ]:
# сколько можно потратить на одну скважину
budgeе_one_point = BUDGET / BEST_POINTS
budgeе_one_point

50000000.0

In [ ]:
# сколько тысяч баррелей должно быть в скважине, чтобы вернуть потраченные деньги на ее разработку

minimum_barrels_point = budgeе_one_point / INCOME_ONE_THOUSAND_BARREL
minimum_barrels_point

111.11111111111111

## Вывод

**В результате расчетов, установлено:**

1. Минимальное необходимое количество баррелей на одной разрабатываемой точке (для ее окупания) должно быть порядка 111 тыс. баррелей.
2. Среднее значение запасов баррелей по точкам в регионах меньше указанного значения (111 тыс. баррелей).
3. Так как, точек в регионах достаточно много (значительно больше 200), а также известно, что максимальные значения запасов в некоторых точках регионов "data_0", "data_1", "data_2" достигаю 185, 138, 190 тыс. баррелей соответственно, то целесообразно предположить, что проект вполне окупаем.

## Расчёт прибыли и рисков 

In [ ]:
# функция выбирает 200 лучших (на основе предсказания модели)
# возвращает пандовскую серию ИСТИННЫХ значений запасов баррелей 
def best_200_target(target_list_subsample):
  # удаление столбцов из признаков
  # назначение целевого признака
  features = target_list_subsample.drop(['id', 'product'], axis=1)
  target = target_list_subsample['product']
  # деление данных на обучающую и валидационную выборки в соотношении 75:25
  features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
  # обучение моли линейной регрессии на данных региона
  # вывод предсказанного среднего запаса сырья
  # вывод метрик качества модели линейной регрессии
  model = LinearRegression()
  model.fit(features_train, target_train)
  predicted_valid = model.predict(features)
  
  predicted_valid = pd.Series(predicted_valid)
  best_200_predict = predicted_valid.sort_values(ascending=False).head(200)
  best_200_target = target[best_200_predict.index]

  return best_200_target

In [ ]:
# функция подсчета прибыли
def profit(list):
  profit = INCOME_ONE_THOUSAND_BARREL * list.sum() - BUDGET

  return profit

In [ ]:
# функция симплит 500 случайных скважин, выбирает среди низ 200 лучших (на основе предсказания модели)
# возвращает 95%-й доверительный интервал, среднюю прибыль, долю рисков на основе ИСТИННЫХ значений запасов баррелей
def sample(data):
  state = np.random.RandomState(12345)

  values = []
  values_risk = []
  for i in range(1000):
    target_list_subsample = data.sample(n=EXPLORE_POINTS, replace=True, random_state=state)
    target_list_subsample = target_list_subsample.reset_index(drop=True)
    #print(target_list_subsample)
    best_target = best_200_target(target_list_subsample)
    #print(best_target)
    prof = profit(best_target)
    if prof < 0:
      values_risk.append(prof)

    values.append(prof)
  values_risk = pd.Series(values_risk)
  values = pd.Series(values)
  
  lower = values.quantile(0.025)
  upper = values.quantile(0.975)
  percent_risk = len(values_risk) / len(values) * 100

  print("95%-й доверительный интервал: от", round(lower / 10**6), "до", round(upper / 10**6), "млн. руб.")
  print("Среднее значение прибыли:", round(mean(values) / 10**6), "млн. руб.")
  print("Доля рисков:", percent_risk, "%")

### Прибыль и риски для региона "data_0"

In [ ]:
sample(data_0)

95%-й доверительный интервал: от -80 до 936 млн. руб.
Среднее значение прибыли: 430 млн. руб.
Доля рисков: 5.2 %


### Прибыль и риски для региона "data_1"

In [ ]:
sample(data_1)

95%-й доверительный интервал: от 33 до 884 млн. руб.
Среднее значение прибыли: 450 млн. руб.
Доля рисков: 1.9 %


### Прибыль и риски для региона "data_2"

In [ ]:
sample(data_2)

95%-й доверительный интервал: от -153 до 877 млн. руб.
Среднее значение прибыли: 377 млн. руб.
Доля рисков: 9.2 %


## Вывод

**Анализ прибыли и рисков регионов, показал:**

*A) В регионе "data_0":*

1. 95%-й доверительный интервал: от -80 до 936 млн. руб.
2. Среднее значение прибыли: 430 млн. руб.
3. Доля рисков: 5.2 %

*Б) В регионе "data_1":*

1. 95%-й доверительный интервал: от 33 до 884 млн. руб.
2. Среднее значение прибыли: 450 млн. руб.
3. Доля рисков: 1.9 %

*В) В регионе "data_2":*

1. 95%-й доверительный интервал: от -153 до 877 млн. руб.
2. Среднее значение прибыли: 377 млн. руб.
3. Доля рисков: 9.2 %


## Общий вывод

**В результате подготовки данных, установлено:**

1. Пропуски отсутствуют.
2. Тип данных во всех столбцах float64, кроме столбца "id" - object.
3. В регионе "data_0" - 10 пар с одинаковыми "id".
4. В регионах "data_1", "data_2" по 4 пары с одинаковыми "id".
5. Так как, прослеживается парность и параметры меняются либо в сторону уменьшения, либо в сторону увеличения, можно предположить, что это результат доразветки скважин. По итогам доразведки уточнялись параметры скважины и ее объём запасов. Исходя из этого, удалим первые (неуточненные данные), а последние записи (по индексу) в соответствующих "id" оставим для дальнейших расчетов.
6. Просмотр описательной статистики показал, что самый большой средний объём запасов в скважинах региона "data_2".
7. Стандартное отклонение в объёмах запасов в скважинах по трем регионам однородно и колеблется в пределах от 44 до 45 тыс. баррелей.
8. Максимальное значение объёма запасов в скважине, также зафиксировано в регионе "data_2" и составляет порядка 190 тыс. баррелей.
9. Аномально высоких или низких значений не обнаружено.
10. Дубликаты в данных не обнаружены.
11. Неявные дубликаты устранены.

**В результате обучения и проверки модели:**

*А) Анализ работы модели линейной регрессии для региона "data_0", показал:*

1. Предсказанный средний запас сырья = 92.6
2. Истинное значение среднего запаса сырья = 92.5
3. Модель демонстрирует хорошие результаты: предсказанное и истинное значения равны при округлении до целых (93=93)
4. RMSE = 37.9

*Б) Анализ работы модели линейной регрессии для региона "data_1", показал:*

1. Предсказанный средний запас сырья = 68.5
2. Истинное значение среднего запаса сырья = 68.8
3. Модель демонстрирует хорошие результаты: предсказанное и истинное значения равны при округлении до целых (69=69)
4. RMSE = 0.9

*В) Анализ работы модели линейной регрессии для региона "data_2", показал:*

1. Предсказанный средний запас сырья = 94.9
2. Истинное значение среднего запаса сырья = 95.0
3. Модель демонстрирует хорошие результаты: предсказанное и истинное значения равны при округлении до целых (95=95)
4. RMSE = 40.2

**В результате расчетов прибыли, установлено:**

1. Минимальное необходимое количество баррелей на одной разрабатываемой точке (для ее окупания) должно быть порядка 111 тыс. баррелей.
2. Среднее значение запасов баррелей по точкам в регионах меньше указанного значения (111 тыс. баррелей).
3. Так как, точек в регионах достаточно много (значительно больше 200), а также известно, что максимальные значения запасов в некоторых точках регионов "data_0", "data_1", "data_2" достигаю 185, 138, 190 тыс. баррелей соответственно, то целесообразно предположить, что проект вполне окупаем.

**Анализ прибыли и рисков регионов, показал:**

*A) В регионе "data_0":*

1. 95%-й доверительный интервал: от -80 до 936 млн. руб.
2. Среднее значение прибыли: 430 млн. руб.
3. Доля рисков: 5.2 %

*Б) В регионе "data_1":*

1. 95%-й доверительный интервал: от 33 до 884 млн. руб.
2. Среднее значение прибыли: 450 млн. руб.
3. Доля рисков: 1.9 %

*В) В регионе "data_2":*

1. 95%-й доверительный интервал: от -153 до 877 млн. руб.
2. Среднее значение прибыли: 377 млн. руб.
3. Доля рисков: 9.2 %

**РЕЗЮМЕ:**

*Наиболее перспективным для разработки скважин является регион "data_1", так как:*

1. Принесет потенциально большую среднюю прибыль: 450 млн. руб.
2. Имеет потенциально меньшее долю рисков: 1.9 %.